In [107]:
import requests
from bs4 import BeautifulSoup
import json
import queue
import time
import os
import pandas as pd
# from dotenv import load_dotenv
from requests.exceptions import ProxyError, HTTPError
import requests


In [108]:
# Fake browser-like headers
BASE_HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
}

# host = 'brd.superproxy.io'
# port = 22225
# username = os.environ['BRD_USERNAME']
# password = os.environ['BRD_PASSWORD']

# proxy_url = f'http://{username}:{password}@{host}:{port}'

# proxies = {
#     'http': proxy_url,
#     'https': proxy_url
# }
# Structure payload.
payload = {
    'source': 'universal_ecommerce',
    'url': 'https://www.walmart.com/reviews/product/720275047'
}

In [109]:
BASE_URL = "https://www.walmart.com"


In [110]:
# List of search queries ,"hoodie","drinking+water"
search_queries = ["Baby+and+Toddler+Music+Toys"]


In [111]:
# Initialize a queue for product URLs and a set for seen URLs
product_queue = queue.Queue()
seen_urls = set()

In [112]:
def get_product_links_from_search_page(query, page_number):
    search_url = f"https://www.walmart.com/search?q={query}&page={page_number}"
    max_retries = 5
    backoff_factor = 3
    for attempt in range(max_retries):
        try:
             # Define the payload. Adjust if necessary.
            payload = {
                'source': 'universal_ecommerce',
                'url': search_url,
                'geo_location': 'United States',
                'locale': 'en-us',
                'render': 'html'
                }

            
            # Get the response.
            response = requests.post(
                'https://realtime.oxylabs.io/v1/queries',
                auth=('Giddy_I7kao', 'PataKazi254_'),  
                json=payload,
            )
             # Convert the response to JSON.
            data = response.json()
            
            # Extract the HTML content from the JSON.
            ht = data['results'][0]['content']
            soup = BeautifulSoup(ht, 'html.parser')
            
            # Check for the script tag that might contain extra data.
            script_tag = soup.find('script', id='__NEXT_DATA__')
            if script_tag is None:
                print("No __NEXT_DATA__ script tag found.")
                return None
            data = json.loads(script_tag.string)

            base_url = data['props']['pageProps']['initialData']['searchResult']['itemStacks'][0]['items']
            product_links = []
            for i in range(len(base_url)):
                if 'usItemId' in base_url[i].keys():
                    full_url = "https://www.walmart.com/reviews/product/" + base_url[i]['usItemId']
                    product_links.append(full_url)

            return product_links

        except ProxyError as e:
            wait_time = backoff_factor ** attempt
            print(f"Proxy error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except HTTPError as e:
            if e.response.status_code == 412:
                print(f"Precondition Failed (412): {e}. Skipping URL.")
                break
            wait_time = backoff_factor ** attempt
            print(f"HTTP error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except Exception as e:
            print(f"Failed to get product links for query: {query} on page: {page_number}. Error: {e}")
            break

    print(f"Skipping query after {max_retries} retries: {query} on page: {page_number}")
    return []

In [113]:
def extract_product_info(product_url):
    print("Processing URL", product_url)
    max_retries = 5
    backoff_factor = 3
    for attempt in range(max_retries):
        try:
        
            # Define the payload. Adjust if necessary.
            payload = {
                'source': 'universal_ecommerce',
                'url': product_url,
                'geo_location': 'United States',
                'locale': 'en-us',
                'render': 'html'
                }

            
            # Get the response.
            response = requests.post(
                'https://realtime.oxylabs.io/v1/queries',
                auth=('Giddy_I7kao', 'PataKazi254_'),  
                json=payload,
            )
            
            # Convert the response to JSON.
            data = response.json()
            
            # Extract the HTML content from the JSON.
            ht = data['results'][0]['content']
            soup = BeautifulSoup(ht, 'html.parser')
            
            # Check for the script tag that might contain extra data.
            script_tag = soup.find('script', id='__NEXT_DATA__')
            if script_tag is None:
                print("No __NEXT_DATA__ script tag found.")
                return None
            data = json.loads(script_tag.string)
            # Instead of response with job status and results url, this will return the
            # JSON response with results.

            initial_data = data["props"]["pageProps"]["initialData"]["data"]
            product_data = initial_data["product"]
            reviews_data = initial_data["reviews"]
            reviews_datas = data['props']['pageProps']['initialData']['data']['reviews']['customerReviews']

            product_info_data = []
            
            for i in range(len(reviews_datas)):
                product_info = {
                    "product_name": product_data["name"],
                    'review_text':reviews_datas[i]['reviewText'],
                    'product_rating': reviews_datas[i]['rating'],
                    'review_date':reviews_datas[i]['reviewSubmissionTime'],
                    "avg_rating": reviews_data.get("averageOverallRating", 0),

                }
                product_info_data.append(product_info)

            return product_info_data

        except ProxyError as e:
            wait_time = backoff_factor ** attempt
            print(f"Proxy error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except HTTPError as e:
            if e.response.status_code == 412:
                print(f"Precondition Failed (412): {e}. Skipping URL.")
                break
            wait_time = backoff_factor ** attempt
            print(f"HTTP error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except Exception as e:
            print(f"Failed to process URL: {product_url}. Error: {e}")
            break

    print(f"Skipping URL after {max_retries} retries: {product_url}")
    return None

In [114]:
# List of product URLs you want to process
product_urls =[] 
for query in search_queries:
    for i in range(1,10):
        product_urls.append(get_product_links_from_search_page(query, i))
# Create a dictionary to store results.
# You can choose the key based on the URL or product id.
all_product_info = []

for urli in product_urls:
    for url in urli:
        # Call extract_product_info for each URL.
        product_info_list = extract_product_info(url)
        # Check if data was returned
        if product_info_list is not None:
            # Option 1: Use the product URL as the key.
            all_product_info.append(product_info_list)
            
            # Option 2: If you prefer to use a unique product id, you could extract that from the result.
            # For example, if the first review dictionary has an "item_id", you can do:
            # product_id = product_info_list[0].get("item_id")
            # all_product_info[product_id] = product_info_list



Processing URL https://www.walmart.com/reviews/product/298189431
Processing URL https://www.walmart.com/reviews/product/5376359346
Processing URL https://www.walmart.com/reviews/product/13687871504
Processing URL https://www.walmart.com/reviews/product/2155696642
Processing URL https://www.walmart.com/reviews/product/298189431
Processing URL https://www.walmart.com/reviews/product/5298196691
Processing URL https://www.walmart.com/reviews/product/925567884
Processing URL https://www.walmart.com/reviews/product/554023453
Processing URL https://www.walmart.com/reviews/product/5232580398
Processing URL https://www.walmart.com/reviews/product/513094341
Processing URL https://www.walmart.com/reviews/product/466689132
Processing URL https://www.walmart.com/reviews/product/5708953764
Processing URL https://www.walmart.com/reviews/product/123184520
Processing URL https://www.walmart.com/reviews/product/1923686498
Processing URL https://www.walmart.com/reviews/product/2589913228
Processing URL h

KeyboardInterrupt: 

In [ ]:
len(product_urls)

0

In [ ]:
all_product_info[0]

[{'product_name': 'Apple MacBook Air 13.3 inch Laptop - Space Gray, M1 Chip, Built for Apple Intelligence, 8GB RAM, 256GB storage',
  'price': 629,
  'item_id': '609040889',
  'review_text': "Came in brand new fancy packaging!  I have been wanting one of these forever but couldn't justify the price . I got the silver so it matched my Chromebook when I work side by side. Operating system takes getting used to if you use. Chromebooks but it's easy enough. (CTRL   F is command F and so on...)",
  'product_rating': 5,
  'review_date': '4/25/2024',
  'avg_rating': 4.7,
  'availability': 'IN_STOCK'},
 {'product_name': 'Apple MacBook Air 13.3 inch Laptop - Space Gray, M1 Chip, Built for Apple Intelligence, 8GB RAM, 256GB storage',
  'price': 629,
  'item_id': '609040889',
  'review_text': "We are an Apple family (iPhones, iPads, MacBooks). My daughter is leaving for college next month, so she won't be able to borrow my wife's MacBook for her schoolwork. I was excited to see Walmart become the

In [115]:
final_data = []
for item in all_product_info:
  for i in item:
    final_data.append(i)


In [116]:
df = pd.DataFrame(final_data)

# Display the first few rows of the DataFrame
df

,product_name,review_text,product_rating,review_date,avg_rating
0,OATHX Baby Music Toys Child Educational Wooden...,Awesome package. Price great. My granddaughter...,5,1/22/2025,4.6
1,OATHX Baby Music Toys Child Educational Wooden...,I got this for my niece who is 2 and her 5 yea...,5,12/14/2024,4.6
2,OATHX Baby Music Toys Child Educational Wooden...,Super Cute!!! My grandkids LOVED it! I didn't ...,5,12/20/2024,4.6
3,OATHX Baby Music Toys Child Educational Wooden...,I purchased this for my 7 month old and she lo...,5,7/14/2024,4.6
4,OATHX Baby Music Toys Child Educational Wooden...,Purchased for my nephews first birthday and he...,5,1/12/2025,4.6
...,...,...,...,...,...
434,Battat Croco Pop Piano Toy Keyboard with Songs...,I love the creativity they designed this with....,5,12/3/2024,4.8
435,Battat Croco Pop Piano Toy Keyboard with Songs...,My 20 month old loves it. He is a wild child ...,5,10/29/2024,4.8
436,Battat Croco Pop Piano Toy Keyboard with Songs...,"I bought this for my 18 month old son, who enj...",4,12/16/2024,4.8
437,Battat Croco Pop Piano Toy Keyboard with Songs...,My daughter who is about to turn two found thi...,1,1/23/2025,4.8


In [117]:
df.to_csv('toy_reviews.csv')